<h3 style="color:red">Code to copy dataset, run only once!!!!</h3>

In [ ]:
""" import os
import shutil

# Define the paths
source_folder = r'D:\testproject\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\valid'
destination_folder = r'D:\testproject\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\validfull'

# Initialize a counter for the total number of files copied
total_files_copied = 0

# Iterate through each subfolder in the source folder
for folder_name in os.listdir(source_folder):
    folder_path = os.path.join(source_folder, folder_name)
    
    # Check if it's a directory
    if os.path.isdir(folder_path):
        # Iterate through each file in the subfolder
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            # Copy the file to the destination folder
            shutil.copy(file_path, destination_folder)
            # Increment the counter
            total_files_copied += 1
            print(f"[{total_files_copied}] Copying {file_name} from {folder_name} to {destination_folder}")

# Display the total number of files copied
print(f"Total files copied: {total_files_copied}")
 """

In [ ]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Constants
train_data_path = "train_data.csv"
valid_data_path = "valid_data.csv"
train_images_folder = r'D:\testproject\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\trainfull'
valid_images_folder = r'D:\testproject\New Plant Diseases Dataset(Augmented)\New Plant Diseases Dataset(Augmented)\validfull'
image_size = (299, 299)
batch_size = 32
epochs = 10

# Load data
train_df = pd.read_csv(train_data_path)
valid_df = pd.read_csv(valid_data_path)

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_images_folder,
    x_col='image_name',
    y_col='image_class',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory=valid_images_folder,
    x_col='image_name',
    y_col='image_class',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Define model architecture
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator,
          steps_per_epoch=train_generator.samples // batch_size,
          epochs=epochs,
          validation_data=valid_generator,
          validation_steps=valid_generator.samples // batch_size)

# Save the model
model.save("leaf_disease_xception_model.h5")
